## Get FarmOS token

In [ ]:
import json
from pyodide import to_js
from IPython.display import JSON
from js import Object, fetch

token_url="https://waterbearcollective.farmos.net/oauth/token"

username="XXXX"
password="XXXX"
full_body = "grant_type=password&username="+username+"&password="+password+"&client_id=farm&scope=farm_manager"
resp = await fetch(token_url,
  method="POST",
body=full_body,  
  headers=Object.fromEntries(to_js({ "Content-Type": "application/x-www-form-urlencoded" })),
)
res = await resp.text()
rj=json.loads(res)
token=rj['access_token']
token_string='Bearer '+rj['access_token']
print(token)

## Retrieve land assets

In [ ]:
url="https://waterbearcollective.farmos.net/api/asset/land"
resp = await fetch(url,
  method="GET",
  headers=Object.fromEntries(to_js({ "Authorization": token_string })),
)
res = await resp.text()
JSON(json.loads(res))


## Create a land asset

In [ ]:
url="https://waterbearcollective.farmos.net/api/asset/land"

json_to_send = {
  "data": {
    "type": "asset--land",
    "attributes": {
     "name": "Wheeee",
     "land_type":"bed",
     "timestamp": "1526584271"
    }
  }
}

resp = await fetch(url,
  method="POST",
   body=json.dumps(json_to_send),
    headers=Object.fromEntries(to_js({ "Authorization": token_string,"Content-Type": "application/vnd.api+json"}))
)
res = await resp.text()
JSON(json.loads(res))



## Retrieve the API schema

In [ ]:
url="https://waterbearcollective.farmos.net/api/schema"

resp = await fetch(url,
  method="GET",
  #body = "grant_type=password&username=donblair&password=farmoscat999&client_id=farm&scope=farm_manager",
  #body=json.dumps({ "some" : "json" }),
  #credentials="same-origin",
                   
  headers=Object.fromEntries(to_js({ "Authorization": token_string })),
  #headers=Object.fromEntries(to_js({ "Content-Type": "application/json" })),
)
res = await resp.text()
JSON(json.loads(res))
